In [1]:
#default_exp snapshots

Outline
- Start solr locally
- Create a collection named "source" with 2 shards and 2 replicas
- Add 3 documents to the collection and commit
- Run backup command to create a snapshot using the curl command against source collection
- Run restore command to restore the snapshot using the curl command against destination collection
- Verify the documents are restored in the destination collection
- [Optional] Delete the collections and stop solr


In [5]:
# Start solr locally
%cd  /home/msivanes/0Work/fusion/4.2.6
!./bin/fusion start

/home/msivanes/0Work/fusion/4.2.6
ln: failed to create symbolic link 'latest/4.2.6': File exists


Starting zookeeper..
Successfully started zookeeper on port 9983 (process ID 576527)
Starting solr....
Successfully started solr on port 8983 (process ID 576602)
Starting api...........................
Successfully started api on port 8765 (process ID 576767)
Starting connectors-classic................
Successfully started connectors-classic on port 8984 (process ID 577813)
Starting connectors-rpc......
Successfully started connectors-rpc on port 8771 (process ID 578410)
Starting proxy.............
Successfully started proxy on port 8764 (process ID 578643)
Starting webapps......
Successfully started webapps on port 8780 (process ID 579255)
Starting admin-ui..................
Successfully started admin-ui on port 8763 (process ID 579548)
Lucidworks Fusion is now available at http://192.168.2.15:8764


In [8]:
# verify solr is running
!./bin/fusion status

zookeeper is running on port 9983 (process ID 584189)
solr is running on port 8983 (process ID 584274)
api is running on port 8765 (process ID 584706)
connectors-classic is running on port 8984 (process ID 585533)
connectors-rpc is running on port 8771 (process ID 586033)
proxy is running on port 8764 (process ID 586165)
webapps is running on port 8780 (process ID 586582)
admin-ui is running on port 8763 (process ID 586738)


In [12]:
# create a collection called 'source' with 2 shards and 2 replicas
import requests

url = 'http://localhost:8983/solr/admin/collections'
params = dict(action='CREATE', name='source', numShards=2, replicationFactor=2, maxShardsPerNode=4)
headers = {'Content-Type': 'application/json'}

resp = requests.post(url, params=params, headers=headers)
if resp.status_code == 200: print('Collection created successfully')
else: print('Error creating collection', resp.text)

Collection created successfully


In [13]:
# add a few documents to the collection
import json, random, requests

url = 'http://localhost:8983/solr/source/update/json/docs'
headers = {'Content-Type': 'application/json'}
docs = [
    {'id': '1', 'name': 'Alice', 'age': random.randint(20, 40), 'gender': 'F'},
    {'id': '2', 'name': 'Bob', 'age': random.randint(20, 40), 'gender': 'M'},
    {'id': '3', 'name': 'Charlie', 'age': random.randint(20, 40), 'gender': 'M'}
]

json_docs = json.dumps(docs)
resp = requests.post(url, headers=headers, data=json_docs)

if resp.status_code == 200: print('Documents added successfully')
else: print('Error adding documents', resp.text)

Documents added successfully


In [17]:
url = 'http://localhost:8983/solr/source/update'
params = {'commit': 'true'}
headers = {'Content-Type': 'application/json'}

response = requests.post(url, params=params, headers=headers)
if response.status_code == 200: print("Commit successful")
else: print("Commit failed", response.text)

Commit successful


In [18]:
# check the documents added exist in the collection
from fastcore.all import urljson
import warnings; warnings.filterwarnings('ignore')
url = 'http://localhost:8983/solr/source/select'
data = urljson(url, {'q': '*:*', 'rows': 10})
data

{'responseHeader': {'zkConnected': True,
  'status': 0,
  'QTime': 26,
  'params': {'q': '*:*', 'rows': '10'}},
 'response': {'numFound': 3,
  'start': 0,
  'maxScore': 1.0,
  'docs': [{'id': '1',
    'name': ['Alice'],
    'age': [32],
    'gender': ['F'],
    '_version_': 1778218855689093120},
   {'id': '2',
    'name': ['Bob'],
    'age': [27],
    'gender': ['M'],
    '_version_': 1778218858419585024},
   {'id': '3',
    'name': ['Charlie'],
    'age': [30],
    'gender': ['M'],
    '_version_': 1778218858425876480}]}}

In [20]:
# check dest collection is empty
url = 'http://localhost:8983/solr/dest/select'
data = urljson(url, {'q': '*:*', 'rows': 10})
data['response']['numFound']

0

In [21]:
# backup the source collection
url = 'http://localhost:8983/solr/admin/collections'
params = dict(action='BACKUP', name='source', collection='source', location='/home/msivanes/0Work/fusion/4.2.6/backups')
resp = requests.post(url, params=params, headers=headers)
if resp.status_code == 200: print('Backup successful')
else: print('Error backing up collection', resp.text)

Backup successful


In [26]:
# restore the dest collection with the snapshot
# Note: The dest collection should not exists. If it exists, delete it first.
url = 'http://localhost:8983/solr/admin/collections'
params = dict(action='RESTORE', name='source', collection='dest', location='/home/msivanes/0Work/fusion/4.2.6/backups')
resp = requests.post(url, params=params, headers=headers)
if resp.status_code == 200: print('Restore successful')
else: print('Error restoring collection', resp.text)

Restore successful


In [27]:
# check if the documents added in source are restored in the dest collection
from fastcore.all import urljson
import warnings; warnings.filterwarnings('ignore')
url = 'http://localhost:8983/solr/dest/select'
data = urljson(url, {'q': '*:*', 'rows': 10})
data

{'responseHeader': {'zkConnected': True,
  'status': 0,
  'QTime': 52,
  'params': {'q': '*:*', 'rows': '10'}},
 'response': {'numFound': 3,
  'start': 0,
  'maxScore': 1.0,
  'docs': [{'id': '1',
    'name': ['Alice'],
    'age': [32],
    'gender': ['F'],
    '_version_': 1778218855689093120},
   {'id': '2',
    'name': ['Bob'],
    'age': [27],
    'gender': ['M'],
    '_version_': 1778218858419585024},
   {'id': '3',
    'name': ['Charlie'],
    'age': [30],
    'gender': ['M'],
    '_version_': 1778218858425876480}]}}

In [24]:
url = 'http://localhost:8983/solr/admin/collections'
params = dict(action='DELETE', name='dest')
resp = requests.post(url, params=params)
if resp.status_code == 200: print('Collection deleted successfully')
else: print('Error deleting collection', resp.text)

Collection deleted successfully
